[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/feiglab/peprnallps/blob/main/PeptideRNALLPS.ipynb)

# Analyzing peptide-RNA liquid-liquid phase separation

This notebook analyzes the energetics of peptide-RNA condensates based on radial distribution functions extracted from coarse-grained simulations with the COCOMO model.

In [ ]:
import os
import sys
import numpy as np
import math
import copy
from pathlib import Path

# set to true if running from command line
cmdline=False

if not cmdline:
    import matplotlib.pyplot as plt

The following cell downloads RDFs from github if they are not available.

In [ ]:
from pathlib import Path
if not Path("RDF").exists():
    print('downloading input structures')
    !mkdir -p RDF
    !npx degit feiglab/peprnallps/RDF RDF  

Class implementing COCOMO interaction potential 

In [ ]:
class Energy:
    def __init__(e,eps,sig,aval,a0):
        e.epsilon=eps          # sqrt(ep1.epsilon*ep2.epsilon)
        e.sigma=sig            # (ep1.sigma+ep2.sigma)/2.0
        e.aval=aval            # (ep1.aval*ep2.aval)
        e.a0=a0                # (ep1.a0+ep2.a0)
        e.kappa=1.0

    def value(e,r):
        sr=(e.sigma/r)
        sr5=sr*sr*sr*sr*sr
        sr10=sr5*sr5
        shortrange=4.0*e.epsilon*(sr10-sr5)
        longrange=(e.aval+e.a0)/r*np.exp(-r/e.kappa)
        return shortrange+longrange

def combineEpsilon(e1,e2):
    return np.sqrt(e1*e2)

def combineSigma(s1,s2):
    return (s1+s2)*0.5

def combineAval(a1,a2):
    return (a1*a2)

def combineA0val(a01,a02):
    return (a01+a02)
    
def sigFromR(r):
    return 2.0*r*2.0**(-1.0/6.0)

Interaction parameters for systems of RNA (poly-Adenine) and peptides (RGRGGn)  

In [ ]:
radius={}
radius['ade']=0.4220*1.2
radius['gly']=0.2617*1.2
radius['arg']=0.3567*1.2

energy={}
energy['ade_ade']=Energy(0.41,
              sigFromR(radius['ade']),
              combineAval(-0.866,-0.866),
              combineA0val(0.05,0.05))

energy['ade_arg']=Energy(combineEpsilon(0.40,0.41)+0.20,
              combineSigma(sigFromR(radius['ade']),sigFromR(radius['arg'])),
              combineAval(-0.866,0.866),
              combineA0val(0.05,0.05))
energy['arg_ade']=energy['ade_arg']

energy['ade_gly']=Energy(0.41,
              combineSigma(sigFromR(radius['ade']),sigFromR(radius['gly'])),
              combineAval(-0.866,0),
              combineA0val(0.05,0))
energy['gly_ade']=energy['ade_gly']

energy['arg_arg']=Energy(0.40,
              sigFromR(radius['arg']),
              combineAval(0.866,0.866),
              combineA0val(0.05,0.05))

energy['arg_gly']=Energy(combineEpsilon(0.40,0.41),
              combineSigma(sigFromR(radius['arg']),sigFromR(radius['gly'])),
              combineAval(0.866,0),
              combineA0val(0.05,0))
energy['gly_arg']=energy['arg_gly']

energy['gly_gly']=Energy(0.41,
              sigFromR(radius['gly']),
              combineAval(0,0),
              combineA0val(0,0))

Example potential for adenine-arginine interactions

In [ ]:
if not cmdline:
    r=np.arange(0.2,5,0.05)
    v=energy['ade_arg'].value(r)

    plt.plot(r,v)
    plt.xlabel('distance [nm]')
    plt.ylabel('energy [kJ/mol]')
    plt.ylim([-1,2])

    plt.show()

Class to manage and read particle density data inside and outside condensates

In [ ]:
class Density:
    def __init__(d,rna,pep,box=100.0,*,nin=None,nout=None,radval=None):
        d.rna=rna
        d.pep=pep
        d.sys=f"a{rna}.rgrgg{pep}"
        d.boxsize=box
        if nin is None or nout is None or radval is None:
            d.loadClusterComponents()
        else:
            d.n={}
            d.n['in']={}
            d.n['in']['ade']=nin[0]
            d.n['in']['arg']=nin[1]
            d.n['in']['gly']=nin[2]
            d.n['out']={}
            d.n['out']['ade']=nout[0]
            d.n['out']['arg']=nout[1]
            d.n['out']['gly']=nout[2]
            d.radius=radval
            
    def loadClusterComponents(d):
        fname=f"RDF/{d.sys}/cluster_components.txt"
        c=np.loadtxt(fname)
        d.n={}
        d.n['in']={}    
        d.n['in']['ade']=c[0]
        d.n['in']['arg']=c[1]
        d.n['in']['gly']=c[2]
        d.n['out']={}
        d.n['out']['ade']=c[3]
        d.n['out']['arg']=c[4]
        d.n['out']['gly']=c[5]
        d.radius=c[6]
        d.totdens=c[7]
                    
    def volume(d,state):
        if state=='in':
            return 4.0*math.pi/3.0*(d.radius*d.radius*d.radius)
        elif state=='out':
            return d.boxsize*d.boxsize*d.boxsize-d.volume('in')
        else:
            print(f'unknown state {state}')
            return 0.0
        
    def sysvol(d):
        return d.boxsize*d.boxsize*d.boxsize
    
    def effvol(d,state):
        cvol=d.volume(state)
        svol=0.0
        for tag in ['ade', 'arg', 'gly']:
            rvol=4.0*math.pi/3.0*(radius[tag]*radius[tag]*radius[tag])
            svol+=rvol*d.n[state][tag]
        return (cvol-svol)

    def molvol(d,component):
        svol=0.0
        if component=='ade':
            svol=4.0*math.pi/3.0*radius['ade']*radius['ade']*radius['ade']*d.rna
        elif component=='pep':
            svol=4.0*math.pi/3.0*(radius['arg']*radius['arg']*radius['arg']*2.0+radius['gly']*radius['gly']*radius['gly']*3.0)*d.pep
        return svol
            
    def ndens(d,tag,state):
        if (d.volume(state)>0):
            return d.n[state][tag]/d.volume(state)
        else:
            return 0.0
        
    def nmol(d,tag,state):
        if tag=='ade':
            return (d.n[state][tag]/d.rna)
        elif tag=='arg':
            return (d.n[state][tag]/(d.pep*2))
        elif tag=='gly':
            return (d.n[state][tag]/(d.pep*3))
        elif tag=='pep':
            return (d.n[state]['arg']/(d.pep*2))
        else:
            return 0.0
    
    def moldens(d,tag,state):
        return d.nmol(tag,state)/d.volume(state)
     
    def sysdens(d,tag):
        return (d.n['in'][tag]+d.n['out'][tag])/d.sysvol()

Class to manage radial distribution functions

In [ ]:
import numpy.polynomial.polynomial as poly

class RDF:
    def __init__(r,rna,pep,state,part1,part2,radval,cut=10.0):
        r.rna=rna
        r.pep=pep
        r.radius=radval
        r.sys=f"a{rna}.rgrgg{pep}"
        r.p1=part1
        r.p2=part2
        r.s=state
        r.rdf=[]
        r.rdfr=[]
        r.rdfavg=[]
        r.loadRDF(cut)
            
    def loadRDF(r,cut=10.0):
        fname=f"RDF/{r.sys}/rdf_{r.s}_{r.p1}_{r.p2}.txt"
        r.rdf=np.loadtxt(fname)
        
        r.rdfr=r.rdf[:,0]
        r.rdfavg=r.rdf[:,1]   
            
        # if cutoff is given, RDF is corrected and normalized to 1 
        if (cut>0):
            gcut=0.0
            for i in range(len(r.rdfr)):
                if (r.rdfr[i]<=cut):
                    gcut=r.rdfavg[i]            
            d=0
            roots=poly.polyroots([0.5*cut*cut*cut,0.0,-3.0/2.0*cut,1.0-gcut])
            for tr in roots:
                if (tr>0 and abs(tr-r.radius*2.0)<abs(d-r.radius*2.0)):
                    d=tr
                    
            for i in range(len(r.rdfr)):
                if (r.rdfr[i]>cut):
                    r.rdfavg[i]=1.0
                else:
                    t=(r.rdfr[i]/d)
                    r.rdfavg[i]/=(1.0-3.0/2.0*t+0.5*(t*t*t))

    def integrate(r,ener,maxr=15.0):
        if (len(r.rdfr)<=0):
            return 0.0
        
        dr=r.rdfr[1]-r.rdfr[0]
        n=int(maxr/dr)
        if (int(n/2)*2!=n):
            n+=1
        v=np.zeros(n+1)
        for i in range(n+1):
            rad=r.rdfr[i]
            if (i>1):
                v[i]=r.rdfavg[i]*ener.value(rad)*rad*rad
        tsum=v[0]
        for i in range(1,n,2):
            tsum+=4.0*v[i]
        for i in range(2,n-1,2):
            tsum+=2.0*v[i]
        tsum+=v[n]
        return (4.0*math.pi*tsum*dr/3.0)

Example for reading condensate RDFs

In [ ]:
if not cmdline:
    dens=Density(20,5)
    #uncorrrected RDF
    rdf_20_5nc=RDF(20,5,'clus','ade','ade',dens.radius,-1)
    #corrected RDF
    rdf_20_5=RDF(20,5,'clus','ade','ade',dens.radius,12)

    plt.plot(rdf_20_5.rdfr,rdf_20_5.rdfavg)
    plt.plot(rdf_20_5.rdfr,rdf_20_5nc.rdfavg)
    plt.xlabel('distance [nm]')
    plt.ylabel('g(r)')
    plt.xlim([0,15])
    plt.show()

Example for reading bulk-phase RDFs

In [ ]:
if not cmdline:
    dens=Density(20,5)
    rdf_20_5_bulk=RDF(20,5,'bulk','ade','ade',dens.radius,-1)

    plt.plot(rdf_20_5_bulk.rdfr,rdf_20_5_bulk.rdfavg)
    plt.xlabel('distance [nm]')
    plt.ylabel('g(r)')
    plt.xlim([0,15])
    plt.show()

Class for putting it all together

In [ ]:
class ClusterEnergy:
    def __init__(c,rna,pep,state,rdfcut=10.0,*,loadrna=None,loadpep=None,altdensity=None):
        c.rna=rna
        c.pep=pep
        c.state=state
        
        if state=='in':
            tag='clus'
        elif state=='out':
            tag='bulk'
        else:
            print('unknown state')

        if loadrna is None: loadrna=rna
        if loadpep is None: loadpep=pep
            
        if altdensity is None:
            c.density=Density(loadrna,loadpep)
        else:
            c.density=altdensity
            
        if (state=='out'):
            rdfcut=-1
        
        c.rdf={}
        c.rdf['ade_ade']=RDF(loadrna,loadpep,tag,'ade','ade',c.density.radius,rdfcut)
        c.rdf['ade_arg']=RDF(loadrna,loadpep,tag,'ade','arg',c.density.radius,rdfcut)
        c.rdf['arg_ade']=c.rdf['ade_arg']
        c.rdf['ade_gly']=RDF(loadrna,loadpep,tag,'ade','gly',c.density.radius,rdfcut)
        c.rdf['gly_ade']=c.rdf['ade_gly']
        c.rdf['arg_arg']=RDF(loadrna,loadpep,tag,'arg','arg',c.density.radius,rdfcut)
        c.rdf['arg_gly']=RDF(loadrna,loadpep,tag,'ade','gly',c.density.radius,rdfcut)
        c.rdf['gly_arg']=c.rdf['arg_gly']
        c.rdf['gly_gly']=RDF(loadrna,loadpep,tag,'gly','gly',c.density.radius,rdfcut)

    def calch(c,component,log=False):
        hsum=0.0
        for o in ['ade', 'arg', 'gly']:
            tag=f"{component}_{o}"
            dens=c.density.ndens(o,c.state)
            h=dens*c.rdf[tag].integrate(energy[tag],15)/2.0
            if (log):
                print(" -> %s %lf" % (o,h))
            hsum+=h
        return hsum

    def calchmol(c,component):
        h=c.calch(component)
        if component=='ade':
            return h*c.rna
        elif component=='arg':
            return h*c.pep*2.0
        elif component=='gly':
            return h*c.pep*3.0
        else:
            return 0.0
    
    def calchn(c,component):
        h=c.calch(component)
        return h*c.density.ndens(component,c.state)

    def calcs(c,component):
        temp=300.0
        kb=0.001987041*4.184
        vratio=c.density.molvol(component)/c.density.sysvol()
        return -temp*kb*np.log(vratio)
    
    def calcsn(c,component):
        s=c.calcs(component)
        return s*c.density.moldens(component,c.state)
    
    def calcsmix(c,component):
        temp=300.0
        kb=0.001987041*4.184
        totdens=c.density.ndens('ade',c.state)/(c.rna)+c.density.ndens('arg',c.state)/(c.pep*2.0)        
        if component=='ade':
            vratio=c.density.ndens('ade',c.state)/(c.rna)/totdens
        elif component=='pep':
            vratio=c.density.ndens('arg',c.state)/(c.pep*2.0)/totdens
        return -temp*kb*np.log(vratio)
    

Example for how to compare chemical potentials between cluster and dilute phases

Chemical potential should be identical if there is phase coexistence. 

There are uncertainties with entropy calculation, RDF approximation, and use of bulk RDFs for dilute phase.    

In [ ]:
print('polyA20 chemical potentials (energy/entropy per molecule)')
for n in [2,3,4,5]:
    tc=ClusterEnergy(20,n,'in',-1)  # use uncorrected RDFs
    tcb=ClusterEnergy(20,n,'out')
    print("RGRGG %2d ade bulk %6.3lf cluster n %6.2lf h %6.3lf s %6.3lf : %6.3lf" % 
          (n,tcb.calch('ade')*tcb.rna,tcb.density.n['out']['ade']/tcb.rna,
           tc.calch('ade')*tc.rna,tc.calcs('ade'),
           tc.calch('ade')*tc.rna+tc.calcs('ade')))
    print("RGRGG %2d pep bulk %6.3lf cluster n %6.2lf h %6.3lf s %6.3lf : %6.3lf" % 
          (n,tcb.calch('arg')*2.0*tcb.pep+tcb.calch('gly')*3.0*tcb.pep,
           tc.density.n['out']['arg']/(2.0*tcb.pep),
           tc.calch('arg')*2.0*tc.pep+tc.calch('gly')*3.0*tc.pep,tc.calcs('pep'),
           tc.calch('arg')*2.0*tc.pep+tc.calch('gly')*3.0*tc.pep+tc.calcs('pep')))
    print()
    

Example for how to estimate total free energy components (enthalpy and entropy) in cluster.

Free energy must be negative for cluster to be stable.

Note uncertainties as mentioned above.

In [ ]:
print('polyA20 total enthalpy and entropy in cluster (multiplied by densities)')
for n in [2,3,4,5]:
    tc=ClusterEnergy(20,n,'in',-1)
    print("rgrgg %2d ade: %1.2lf arg: %1.2lf gly: %1.2lf htot: %1.2lf s: rna %1.2lf pep %1.2lf stot: %1.2lf" % 
      (n,tc.calchn('ade'),tc.calchn('arg'),tc.calchn('gly'),
         tc.calchn('ade')+tc.calchn('arg')+tc.calchn('gly'),
         tc.calcsn('ade'),tc.calcsn('pep'),
         tc.calcsn('ade')+tc.calcsn('pep')))

Free energy as a function of RNA and peptide fractions in cluster estimated using RDFs and density for polyA20/RGRGG_5

This example shows how to estimate energies for other systems for which no RDFs are available.

In [ ]:
def totalEnergy(nrna,npep,nade,npepu,x,y,r):
    nin=[nade*x,npepu*2*y,npepu*3*y]
    nout=[nade*(1.0-x),npepu*2*(1.0-y),npepu*3*(1.0-y)]
    dens=Density(nrna,npep,nin=nin,nout=nout,radval=r)
    ce=ClusterEnergy(nrna,npep,'in',-1,loadrna=20,loadpep=5,altdensity=dens)
    return (ce.calchn('ade')+ce.calchn('arg')+ce.calchn('gly')+ce.calcsn('ade')+ce.calcsn('pep'))    

if not cmdline:
    import matplotlib.colors as colors
   
    d=Density(20,5)
    nade=(d.n['in']['ade']+d.n['out']['ade'])
    npepu=(d.n['in']['arg']+d.n['out']['arg'])/(2.0)
    
    nrna=10
    npep=2

    afrac=np.linspace(0.2,1.0,50)
    pfrac=np.linspace(0.2,1.0,50)
    X,Y=np.meshgrid(afrac,pfrac)
    vfunc=np.vectorize(totalEnergy)
    Z=totalEnergy(nrna,npep,nade,npepu,X,Y,d.radius)

    cp=plt.contourf(X,Y,Z,levels=50,cmap='PRGn',norm=colors.Normalize(vmin=-3,vmax=3))

    plt.colorbar(cp)
    plt.show()